<a href="https://colab.research.google.com/github/shjoo0407/political_disposition/blob/main/eun/Doc2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd /content/drive/MyDrive/2023/SW공모전

/content/drive/MyDrive/2023/SW공모전


In [ ]:
import pandas as pd

In [ ]:
# 형태소 분석기 Mecab 설치
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

In [ ]:
df1 = pd.read_csv('data/final_NPP.csv',  sep=',')
df1 = df1.dropna()

df2 = pd.read_csv('data/TMP_on_process.csv',  sep=',')
df2 = df2.dropna()

In [ ]:
df1['target'] = '국민의힘'
df2['target'] = '민주당'

In [ ]:
df = pd.concat([df1, df2])
df = df[['content','target']]

In [ ]:
print(df[df['target']=='국민의힘'].shape)
print(df[df['target']=='민주당'].shape)

(35400, 2)
(43199, 2)


In [ ]:
df.head(3)

,content,target
0,"김기현 당 대표는 2023. 7. 19.(수) 11:00, 신월 빗물저류배수시설에 ...",국민의힘
1,"오늘 한 언론에 보도된 무소속 박완주 의원의 검찰 공소장 안에는, 강력 성범죄자의 ...",국민의힘
2,7월 19일 원내대책회의 주요내용은 다음과 같다. <윤재옥 원내대표> 전국적인 ...,국민의힘


In [ ]:
df.isna().sum()

content    0
target     0
dtype: int64

In [ ]:
import re

def pre_process(text):
    if isinstance(text, str):  # Check if the input is a string
        # Replace HTML tags with space
        text = re.sub(r"<.*?>", " ", text)  # Escaped the question mark here
        # Remove special characters and digits
        text = re.sub(r"(\d|\W)+", " ", text)

    return text

df['content'] = df['content'].apply(lambda x: pre_process(x))

In [ ]:
mecab = Mecab()

tagged_corpus_list = []

for index, row in tqdm(df.iterrows(), total=len(df)):
  text = row['content']
  tag = row['target']
  tagged_corpus_list.append(TaggedDocument(tags=[tag], words=mecab.morphs(text)))

print('문서의 수 :', len(tagged_corpus_list))

100%|██████████| 78599/78599 [05:48<00:00, 225.76it/s]

문서의 수 : 78599


In [ ]:
from gensim.models import doc2vec
import gensim.models.doc2vec as doc2vec

In [ ]:
# Create and train the Doc2Vec model
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)
# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)
# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)
# 모델 저장
model.save('dart.doc2vec')

In [ ]:
data = pd.read_csv('17대 민주당_보수당 구분.csv',  sep=',')
data = data.dropna()

In [ ]:
data.head()

,L|R,Content
0,0,그러면 회의를 주재토록 하겠습니다. 국회법에 따라 위원장이 선임될 때까지 본 위원이...
1,0,의사일정 제1항 위원장 선임의 건을 상정토록 하겠습니다. 특별위원회의 위원장 선임은...
2,1,위원장님!
3,0,곽성문 위원님 말씀해 주십시오.
4,1,"대구 중․남구 출신 곽성문 위원입니다."" 육상선수권대회유치특별위원회 위원 중에 최다..."


In [ ]:
data = data[:10]

In [ ]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pandas as pd


# 속기록 벡터화 함수
def vectorize_records(records):
    tagged_docs = [TaggedDocument(words=gensim.utils.simple_preprocess(record), tags=[index]) for index, record in enumerate(records)]
    model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4)
    model.build_vocab(tagged_docs)
    model.train(tagged_docs, total_examples=model.corpus_count, epochs=100)
    return model

# 정당 A의 속기록 벡터화
partyA_records = data[data['L|R'] == '1']['content'].tolist()
model_partyA = vectorize_records(partyA_records)

# 정당 B의 속기록 벡터화
partyB_records = data[data['L|R'] == '0']['content'].tolist()
model_partyB = vectorize_records(partyB_records)

# 특정 속기록과 정치적 편향 유사도 계산
def calculate_similarity(model, target_record):
    target_vector = model.infer_vector(gensim.utils.simple_preprocess(target_record))
    most_similar_records = model.docvecs.most_similar([target_vector], topn=5)
    return most_similar_records

# 예시: 특정 속기록과 정당A의 속기록 유사도 계산
target_record = "특정 속기록 내용"
similar_records_partyA = calculate_similarity(model_partyA, target_record)
print("Most similar records in Party A:")
for record_idx, similarity in similar_records_partyA:
    print(f"Record index: {record_idx}, Similarity: {similarity}")

# 예시: 특정 속기록과 정당B의 속기록 유사도 계산
similar_records_partyB = calculate_similarity(model_partyB, target_record)
print("Most similar records in Party B:")
for record_idx, similarity in similar_records_partyB:
    print(f"Record index: {record_idx}, Similarity: {similarity}")